In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
import io
from PIL import Image
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
from sklearn.utils import shuffle
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
x_train=[]
y_train=[]


labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Training',labels[0])
for i in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,i))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[0])

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Training',labels[1])
for j in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,j))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[1])

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Training',labels[2])
for k in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,k))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[2])

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Training',labels[3])
for l in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,l))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[3])



#testing datas

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Testing',labels[0])
for i in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,i))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[0])

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Testing',labels[1])
for j in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,j))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[1])

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Testing',labels[2])
for k in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,k))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[2])

folderpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/bigMRI/Testing',labels[3])
for l in (os.listdir(folderpath)):
  img=cv2.imread(os.path.join(folderpath,l))
  img = cv2.resize(img,(224, 224))
  x_train.append(img)
  y_train.append(labels[3])

In [ ]:
len(x_train)

3264

In [ ]:
x_train[0].shape

(224, 224, 3)

In [ ]:
len(y_train)

3264

In [ ]:
y_train[1]

'glioma_tumor'

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:

x_train, y_train = shuffle(x_train,y_train,random_state=32)

In [ ]:
y_train.shape

(3264,)

In [ ]:
x_train.shape

(3264, 224, 224, 3)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)
x_train.shape

(3264, 224, 224, 3)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_train,y_train, test_size=0.1,random_state=42)

In [ ]:

y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
resnetmodel=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94773248/94765736 [==============================] - 0s 0us/step


In [ ]:
model = resnetmodel.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=resnetmodel.input, outputs = model)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("resnet50.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)


In [ ]:
history = model.fit(X_train,y_train,validation_split=0.1, epochs =12, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/12
83/83 [==============================] - 29s 346ms/step - loss: 0.0034 - accuracy: 0.9992 - val_loss: 0.1634 - val_accuracy: 0.9592

Epoch 00001: val_accuracy improved from -inf to 0.95918, saving model to resnet50.h5
Epoch 2/12
83/83 [==============================] - 28s 338ms/step - loss: 0.0036 - accuracy: 0.9996 - val_loss: 0.1629 - val_accuracy: 0.9626

Epoch 00002: val_accuracy improved from 0.95918 to 0.96259, saving model to resnet50.h5
Epoch 3/12
83/83 [==============================] - 28s 334ms/step - loss: 0.0033 - accuracy: 0.9996 - val_loss: 0.1521 - val_accuracy: 0.9660

Epoch 00003: val_accuracy improved from 0.96259 to 0.96599, saving model to resnet50.h5
Epoch 4/12
83/83 [==============================] - 28s 333ms/step - loss: 0.0032 - accuracy: 0.9996 - val_loss: 0.1387 - val_accuracy: 0.9694

Epoch 00004: val_accuracy improved from 0.96599 to 0.96939, saving model to resnet50.h5
Epoch 5/12
83/83 [==============================] - 28s 338ms/step - loss: 